Industry Classification / Data Exploration

In [55]:
import pandas as pd

labeled = pd.read_csv('../data/train.csv')

full = pd.read_csv('../data/customers.csv')  


In [58]:
labeled.head()

,customer_id,Industry Label (SIC),display_name,keywords
0,649df00d04a03606c70c38d4,Education,Eict Institute Limited,"[ ""Industrial artisans training "", ""Skills dev..."
1,64c25dbe215a3344ed1adc26,Information & Communication,SwiftXR 3D AR VR,"[ ""3D "", ""Augmented Reality "", ""Virtual Realit..."
2,64c2805a53a41a3b816cc3c5,Wholesale & Retail Trade,Zamar Solutions,"[ ""Battery in lagos "", ""Inverter in lagos "", ""..."
3,64c285a4775bb52fd268c188,Wholesale & Retail Trade,Afiari,"[ ""Groceries "", ""Supermarket "", ""Food delivery..."
4,64c3b14d8d37c1784d601049,Health & Social Work,Africa Medical Marketplace,"[ ""Medical Equipment Africa "", ""Healthcare Sup..."


In [59]:
full.head()

,customer_id,display_name,keywords
0,649df00d04a03606c70c38d4,Eict Institute Limited,"[ ""Industrial artisans training "", ""Skills dev..."
1,64c25dbe215a3344ed1adc26,SwiftXR 3D AR VR,"[ ""3D "", ""Augmented Reality "", ""Virtual Realit..."
2,64c2805a53a41a3b816cc3c5,Zamar Solutions,"[ ""Battery in lagos "", ""Inverter in lagos "", ""..."
3,64c285a4775bb52fd268c188,Afiari,"[ ""Groceries "", ""Supermarket "", ""Food delivery..."
4,64c3b14d8d37c1784d601049,Africa Medical Marketplace,"[ ""Medical Equipment Africa "", ""Healthcare Sup..."


In [60]:
for df in [labeled, full]:
    df['display_name'] = df['display_name'].fillna('')
    df['keywords'] = df['keywords'].fillna('[]')
    df['keywords_clean'] = df['keywords'].astype(str).str.strip("[]").str.replace('"', '').str.replace("'", '')
    df['text'] = df['display_name'] + ' ' + df['keywords_clean']

In [68]:
labeled = labeled.rename(columns={'Industry Label (SIC)': 'industry'})

In [69]:
labeled = labeled.dropna(subset=['industry']).copy()
labeled['industry_encoded'] = le.fit_transform(labeled['industry'])


In [70]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(
    labeled['text'], labeled['industry_encoded'], test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train.fillna(''))
X_test_vec = vectorizer.transform(X_test.fillna(''))

In [71]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000)

In [72]:
from sklearn.metrics import classification_report

y_test_clean = labeled.loc[X_test.index, 'industry_encoded']
y_pred = model.predict(X_test_vec)

print(classification_report(
    y_test_clean, y_pred,
    target_names=le.classes_,
    labels=le.transform(le.classes_),
    zero_division=0
))

                                 precision    recall  f1-score   support

  Accommodation & Food Services       0.00      0.00      0.00         0
        Administrative Services       0.00      0.00      0.00         1
Agriculture, Forestry & Fishing       0.00      0.00      0.00         3
           Arts & Entertainment       0.00      0.00      0.00         7
                   Construction       1.00      0.25      0.40         4
                      Education       0.00      0.00      0.00         2
 Financial & Insurance Services       0.00      0.00      0.00         1
           Health & Social Work       0.00      0.00      0.00         4
    Information & Communication       0.63      0.77      0.69        22
                  Manufacturing       0.00      0.00      0.00         7
                 Other Services       0.00      0.00      0.00         3
          Professional Services       1.00      0.17      0.29        12
         Real Estate Activities       0.00      0.

In [73]:
X_full = full['text'].fillna('')
X_full_vec = vectorizer.transform(X_full)
full['industry_predicted'] = le.inverse_transform(model.predict(X_full_vec))

In [74]:
full

,customer_id,display_name,keywords,keywords_clean,text,industry_predicted
0,649df00d04a03606c70c38d4,Eict Institute Limited,"[ ""Industrial artisans training "", ""Skills dev...","Industrial artisans training , Skills develop...",Eict Institute Limited Industrial artisans t...,Wholesale & Retail Trade
1,64c25dbe215a3344ed1adc26,SwiftXR 3D AR VR,"[ ""3D "", ""Augmented Reality "", ""Virtual Realit...","3D , Augmented Reality , Virtual Reality , AR...","SwiftXR 3D AR VR 3D , Augmented Reality , Vi...",Information & Communication
2,64c2805a53a41a3b816cc3c5,Zamar Solutions,"[ ""Battery in lagos "", ""Inverter in lagos "", ""...","Battery in lagos , Inverter in lagos , Solar ...","Zamar Solutions Battery in lagos , Inverter ...",Wholesale & Retail Trade
3,64c285a4775bb52fd268c188,Afiari,"[ ""Groceries "", ""Supermarket "", ""Food delivery...","Groceries , Supermarket , Food delivery , Sho...","Afiari Groceries , Supermarket , Food delive...",Wholesale & Retail Trade
4,64c3b14d8d37c1784d601049,Africa Medical Marketplace,"[ ""Medical Equipment Africa "", ""Healthcare Sup...","Medical Equipment Africa , Healthcare Supplie...",Africa Medical Marketplace Medical Equipment...,Wholesale & Retail Trade
...,...,...,...,...,...,...
796,680ce1b652836a04390d4ada,Smart Bookings Ltd,"[ ""Hotel booking Nigeria "", ""Book hotels onlin...","Hotel booking Nigeria , Book hotels online Ni...","Smart Bookings Ltd Hotel booking Nigeria , B...",Wholesale & Retail Trade
797,6824312c4cb7a54e38389dab,Inospace,"[ ""Micro warehouses "", ""Ecommerce warehousing ...","Micro warehouses , Ecommerce warehousing , La...","Inospace Micro warehouses , Ecommerce wareho...",Other Services
798,6824619bea39312fd0285455,Get Artsy,"[ ""Where can I take mosaic art classes near me...","Where can I take mosaic art classes near me ,...",Get Artsy Where can I take mosaic art classe...,Wholesale & Retail Trade
799,681a0987818a1817495c13c7,Magharibi Greenhouses,"[ ""Greenhouse sale in kenya "", ""Profitable far...","Greenhouse sale in kenya , Profitable farming...",Magharibi Greenhouses Greenhouse sale in ken...,Wholesale & Retail Trade
